In [1]:
# Import relevant libraries
import pandas as pd
import numpy as np
import re
import nltk
import os
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
from nltk.util import ngrams
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from psutil import virtual_memory
from lxml import etree
import glob, csv
import seaborn as sns
from gensim.models import KeyedVectors

In [2]:
# Get Data, clean and prepare for EDA, mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)
%cd /content/gdrive/MyDrive/STOR565_Final/datasets

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Mounted at /content/gdrive
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-25-1ffc12010630>", line 4, in <cell line: 4>
    get_ipython().run_line_magic('cd', '/content/gdrive/MyDrive/STOR565_Final/datasets')
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py", line 342, in cd
    oldcwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local

In [3]:
# Loading data, previews
raw_text = pd.read_csv("raw_data_TDAVIDSON.csv")
raw_text.info()
raw_text.dtypes

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-25-91bb7506bc9b>", line 2, in <cell line: 2>
    raw_text = pd.read_csv("raw_data_TDAVIDSON.csv")
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 912, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 577, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 1407, in __init__
    self._engine = self._make_engine(f, self.engine)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 1661, in _make_engine
    self.handles = get_handle(
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/common.py", line 859, in get_handle


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-25-91bb7506bc9b>", line 2, in <cell line: 2>
    raw_text = pd.read_csv("raw_data_TDAVIDSON.csv")
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 912, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 577, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 1407, in __init__
    self._engine = self._make_engine(f, self.engine)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 1661, in _make_engine
    self.handles = get_handle(
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/common.py", line 859, in get_handle


In [ ]:
nltk.download('stopwords')

# Function to clean tweets
def clean_tweet(tweet):
    tweet = tweet.lower() #Lowercase
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)     # Remove URLs
    tweet = re.sub(r'\d+', '', tweet)  # Remove numbers
    tweet = re.sub(r'[^\w\s\']', '', tweet)  # Remove special characters except apostrophes
    tweet = re.sub(r'\s+', ' ', tweet).strip()     # Remove extra whitespace
    tweet = re.sub(r'rt', ' ', tweet).strip() # Remove "rt"
    tweet = re.sub(r'@[^\s]+', '', tweet)  # Remove user handles
    tweet_tokens = tweet.split()  # Split tweet into words
    tweet = ' '.join([word for word in tweet_tokens if word not in stopwords.words('english')])
    return tweet

# Apply cleaning function to tweets
if 'tweet' in raw_text.columns:
    raw_text['clean_tweet'] = raw_text['tweet'].apply(clean_tweet)
    raw_text = raw_text.rename(columns={'tweet': 'og_tweet'})

raw_text['tweet_length'] = raw_text['clean_tweet'].apply(len)

# Display the first few rows to verify the cleaning
print(f'Number of entries after cleaning: {raw_text.shape[0]}')

#raw_text.head(25)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
raw_text = raw_text.iloc[:500]

In [ ]:
# 0. Check how much RAM we have available
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:
# Convert data frame to list
tweet_data = raw_text['clean_tweet'].tolist()

# Tokenize the text data
tweet_tokens = [tweet.split() for tweet in tweet_data]

# Train the Word2Vec model
word2vec_model = Word2Vec(sentences=tweet_tokens, vector_size=300, window=5, min_count=1, seed=42)

# Save the trained Word2Vec model
word2vec_model.save("word2vec_model.bin")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gensim/utils.py", line 764, in save
    _pickle.dump(self, fname_or_handle, protocol=pickle_protocol)
TypeError: file must have a 'write' attribute

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-46ccbf59a5c0>", line 11, in <cell line: 11>
    word2vec_model.save("word2vec_model.bin")
  File "/usr/local/lib/python3.10/dist-packages/gensim/models/word2vec.py", line 1923, in save
    super(Word2Vec, self).save(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/gensim/utils.py", line 767, in save
    self._smart_save(fname_or_handle, separately, sep_limit, ignore, pickle_protocol=pickle_protocol)
  File "/usr/local/lib/python3.10/dist-packages/gensim/utils

In [ ]:
## Load the GloVe model (make sure you've downloaded it or convert it to Word2Vec format)
#! wget http://nlp.stanford.edu/data/glove.twitter.27B.zip
#! unzip glove.twitter.27B.zip


# glove_file = 'glove.twitter.27B.50d.txt'
# glove_vectors = KeyedVectors.load_word2vec_format(glove_file)


ValueError: invalid literal for int() with base 10: '<user>'

In [ ]:

def convert_glove_to_word2vec(glove_input_file, word2vec_output_file):
    # Count the number of lines (words) in the GloVe file
    num_lines = sum(1 for line in open(glove_input_file, 'r', encoding='utf-8'))

    # Read the dimensions from the first line
    with open(glove_input_file, 'r', encoding='utf-8') as f:
        dimensions = len(f.readline().split()) - 1

    # Write the header and the GloVe vectors to the new word2vec file
    with open(word2vec_output_file, 'w', encoding='utf-8') as outfile:
        outfile.write(f"{num_lines} {dimensions}\n")
        with open(glove_input_file, 'r', encoding='utf-8') as infile:
            for line in infile:
                outfile.write(line)

# Path to the GloVe file
glove_input_file = 'glove.twitter.27B.50d.txt'
word2vec_output_file = 'glove.twitter.27B.50d.w2vformat.txt'

# Convert the file
convert_glove_to_word2vec(glove_input_file, word2vec_output_file)


Word Embeddings with Word2Vec

In [ ]:
# 4. Split the cleaned description into word tokens
corpus = []
for row in raw_text['clean_tweet']:
    corpus.append(row.split())

In [ ]:
# Load the converted GloVe model in word2vec format
word2vec_model = KeyedVectors.load_word2vec_format('glove.twitter.27B.50d.w2vformat.txt', binary=False)

# Load pre-trained GloVe model
pretrained_vectors = KeyedVectors.load_word2vec_format('glove.twitter.27B.50d.w2vformat.txt', binary=False)

# Initialize Word2Vec with the loaded vectors
word2vec_model = Word2Vec(size=50, min_count=1)  # ensure the size matches GloVe dimension
word2vec_model.build_vocab([list(pretrained_vectors.key_to_index.keys())])
word2vec_model.wv.vectors[:] = pretrained_vectors.vectors

# Lock the vectors to prevent them from being updated during training
word2vec_model.wv.vectors_lockf = np.zeros(len(word2vec_model.wv.vectors), dtype=np.float32)

# Build vocabulary from your corpus (assuming 'corpus' is a list of tokenized texts)
word2vec_model.build_vocab(corpus, update=True)

# Fine-tune the model on your corpus
word2vec_model.train(corpus, total_examples=len(corpus), epochs=5)


TypeError: Word2Vec.__init__() got an unexpected keyword argument 'size'

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Get word vectors from the trained Word2Vec model for the first tweet
first_tweet_word_vectors = [word2vec_model[token] for token in tweet_tokens[0] if token in word2vec_model]

# Convert the list of word vectors for the first tweet into a numpy array
first_tweet_word_vectors = np.array(first_tweet_word_vectors)

# Pad sequences to ensure they all have the same length
first_tweet_word_vectors = pad_sequences([first_tweet_word_vectors], dtype='float32', padding='post')

# Show how long the vectors are
print(f"Dimensions: {first_tweet_word_vectors.shape}")

# Show the first 10 "columns" for the word 'man' in the first tweet
print(f"First 10 dimensions of the word embedding for first word in the first tweet: {first_tweet_word_vectors[0, :10]}")

# Reshape the array to 2 dimensions
reshaped_word_vectors = first_tweet_word_vectors.reshape(first_tweet_word_vectors.shape[1], first_tweet_word_vectors.shape[2])

# Visualize Word Vectors for the first tweet
#sns.set(font_scale=2)
#sns.set_style({"ytick.color": "red"})
#f, ax = plt.subplots(figsize=(40, 10))
#sns.heatmap(reshaped_word_vectors, cmap='coolwarm', xticklabels=False, yticklabels=tweet_tokens, cbar=False, robust=True, linewidth=.5)
#plt.show()



Dimensions: (1, 9, 50)
First 10 dimensions of the word embedding for first word in the first tweet: [[-6.8344e-01 -1.5732e-01  2.0477e-01  5.1348e-01 -4.4520e-01  2.4924e-01
   9.0365e-01  2.3955e-01 -4.6274e-01  5.5980e-01 -1.1015e+00 -1.2249e-01
  -4.2492e+00  4.1175e-02  7.4230e-01  5.5151e-01 -4.1298e-02 -4.6402e-01
   8.7095e-01 -5.2454e-01  3.8639e-01 -1.9028e-01 -6.8765e-02 -3.9335e-01
   5.7378e-01  1.0176e+00 -2.4217e-01  3.2965e-02 -1.3523e-01 -1.5935e-01
  -2.0372e-01 -2.9916e-01 -9.1943e-03 -4.8512e-02  3.1916e-01  6.6313e-01
   9.7535e-02  4.8647e-01  1.0198e-01 -1.5611e+00 -6.5179e-01  6.0939e-02
   1.7976e-01  7.3439e-01  3.7133e-01 -2.5663e-01  9.7130e-01  5.2974e-01
  -3.0886e-01 -5.4252e-02]
 [ 9.3968e-01  6.1427e-01 -5.0373e-02  2.0524e-01 -7.6340e-01  6.8479e-01
   1.2448e+00 -1.7563e-01  7.8323e-02  1.0210e+00 -4.2169e-01 -7.4699e-01
  -3.2445e+00  1.0047e-01 -5.1164e-01  8.2005e-01 -1.7838e-01 -9.0834e-01
   2.5657e-01 -5.9778e-01  3.1929e-01 -6.7332e-01 -3.4205e-

In [ ]:
# 2. Calculate Cosine Similarity for first word (man) - optional
cos_sim = cosine_similarity(reshaped_word_vectors, np.tile(reshaped_word_vectors[0], (7, 1)))[:, 1]

# 3. Generate Cosine Similarity Matrix
sim_matrix = cosine_similarity(reshaped_word_vectors, reshaped_word_vectors)

# 4. Get actual words
word_labels = [str(word) for word in tweet_tokens]

# 5. Generate Heat Map
sns.set(font_scale=1)
sns.heatmap(sim_matrix, annot=True, xticklabels=word_labels, yticklabels=word_labels, cmap="YlGnBu")

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7d63d953c3a0> (for post_execute):


KeyboardInterrupt: 

In [ ]:
#Dimensionality Reduction with t-SNE

In [ ]:
# 1. Import required package
from sklearn.manifold import TSNE

# 2. Set value for perplexity: we have very few objects (i.e., words), therefore we set a very low value
perp = 3  # perplexity of similarity kernel

# 3. Set multiple Seeds for random initializations
sds=[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
KL = []

# 4. Find best seed (i.e., lowest KL divergence)
for s in sds:
    tsne = TSNE(n_components = 4, perplexity = perp, init='random', learning_rate=100, random_state=s)
    X_tsne = tsne.fit_transform(np.array(first_tweet_word_vectors))
    KL.append(tsne.kl_divergence_)
    print(tsne.kl_divergence_, end=" ")

# 5. Fit Best Model
Y = TSNE(n_components = 2, perplexity = perp, init='random', learning_rate=100, verbose=1, random_state=sds[KL.index(min(KL))]).fit_transform(np.array(first_tweet_word_vectors))

ValueError: perplexity must be less than n_samples

In [ ]:
# 6. Free up some RAM by deleting what we don't need
word_vectors=[]
sim_matrix=[]
word_labels=[]
cos_sim=[]
base=[]
df=[]
tSNE=[]
Y=[]
KL=[]
ax=[]

Word Embeddings with Word2Vec